In [89]:
import re
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [90]:
# Carga de datos
import pandas as pd

df_repuestos = pd.read_excel(r"C:\Users\danie\OneDrive\IA\Proyecto_Final_IA\data\Lineas de Mantenimiento - repuestos.xlsx")
df_mant = pd.read_excel(r"C:\Users\danie\OneDrive\IA\Proyecto_Final_IA\data\INFORME DE ESTADO DE COMUNICACIONES NO COMUNICACIONES E INFORMATICA ( Br Bl I).xlsx",header=3)

In [91]:
# Mostrar los datos
df_repuestos.head()

,EFECTO (NNE),TIPO DE FALLA,RESEÑA DE LA FALLA,REPUESTOS MAS PROBABLES,REPUESTOS CASI PROBABLES,REPUESTOS POCO PROBABLES
0,MOTOROLA 5050,rx,El equipo transmite pero no recibe,modulo,transistor pnp,NaN
1,MOTOROLA 5050,rx,El equipo transmite pero no recibe,PTT,NaN,NaN
2,MOTOROLA 5050,rx,El equipo transmite pero no recibe,antena,NaN,NaN
3,MOTOROLA 5050,no enciende,El equipo no enciende,placa de alimentacion,bateria,NaN
4,MOTOROLA 5050,no enciende,El equipo no enciende,pin de carga,NaN,NaN


**Tratamiento de los datos**
a) Borrar los los nombres de las columnas invalidas
B) Reemplazar los NaN por 0 - menos la ultima columna


In [92]:
# reemplazar las columnas no validas por la última columna válida anterior
new_cols = []
for i, col in enumerate(df_mant.columns):
    if "Unnamed" in col:
                new_cols.append(new_cols[-1])
    else:
        new_cols.append(col)

df_mant.columns = new_cols

# Eliminos los NaN todas menos la última
df_mant.iloc[:, :-1] = df_mant.iloc[:, :-1].fillna(0)

## MUESTRA DEL DF

In [93]:
df_mant.head()

,N° Ord,NNE,INE,BAL TANDIL,BAL TANDIL,BAL TANDIL,ESC COM BL 1,ESC COM BL 1,ESC COM BL 1,CDO BR BL I,...,JEF AGR AA EJ 601 - EC,JEF AGR AA EJ 601 - EC,JEF AGR AA EJ 601 - EC,G MANT SIS AA 601,G MANT SIS AA 601,G MANT SIS AA 601,TOTAL GENERAL,TOTAL GENERAL,TOTAL GENERAL,INDIQUE FALLAS
0,0,0,0,Cargo,E/S,F/S,Cargo,E/S,F/S,Cargo,...,Cargo,E/S,F/S,Cargo,E/S,F/S,Cargo,E/S,F/S,PRINCIPALES
1,1,3895002526898,EQ DE TEN Y REPL RL 31 P/CABLE TEF CAMP,0,0,0,7,7,0,0,...,0,0,0,0,0,0,8,8,0,NaN
2,2,3895003563937,EJE DE CARRETE DE TEN Y REPL RL 27 D P/CABLE T...,0,0,0,7,7,0,0,...,0,0,0,0,0,0,7,7,0,NaN
3,3,3895005377953,EQ DE TEN Y REPL RL 26 P/CABLE TEF CAMP,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,1,UM (CUENTA MUSEO ) (ESC COM BL 1)
4,4,5805005211320,TELEFONO DE CAMPAÑA PORTATIL TA 1/PT,0,0,0,11,0,11,0,...,0,0,0,0,0,0,11,0,11,UM (CUENTA MUSEO ) (ESC COM BL 1)


## IDENTIFICAR LAS UNIDADES APOYADAS

In [94]:
# Columnas a excluir
excluir = ['N° Ord', 'NNE', 'INE', 'TOTAL GENERAL', 'INDIQUE FALLAS']

# Columnas únicas sin las excluidas
cols_filtradas = df_mant.columns.drop_duplicates().drop(excluir)

# Mostrar en columnas verticales
#Unidades = df_mant[cols_filtradas].nunique()
#print(Unidades)
print("Elementos Militares:")
for col in cols_filtradas:
    print(col)  

Elementos Militares:
BAL TANDIL
ESC COM BL 1
CDO BR BL I
ESC ICIA BL 1
RC TAN 10
GA BL 1
RC TAN 2
ESC ING BL 1
RI MEC 7
ESC EXPL C BL 1
RC TAN 8
GAA MIX 602
GAA 601
JEF AGR AA EJ 601 - EC
G MANT SIS AA 601


### TRATAMIENTO DEL DF PARA SER PROCESADO

In [95]:
# Usar la primera fila como encabezado y eliminar los nombres originales
df_fallas = df_mant[['NNE', 'INE', 'TOTAL GENERAL', 'INDIQUE FALLAS']]
df_fallas.columns = df_fallas.iloc[0] # Elimino la fila 0
df_fallas = df_fallas.iloc[:-2] # Elimino las dos últimas filas
df_fallas = df_fallas[1:]   # Elimino la fila 1

# Cargo los nombres de las columnas
df_fallas.columns.values[0] = "NNE"
df_fallas.columns.values[1] = "INE"
df_fallas.columns.values[5] = "PRINCIPALES FALLAS"
filas=df_fallas.shape[0]
#df_fallas.head(40)

In [96]:
# Error de carga de datos

for idx, row in df_fallas.iterrows():
    if (row["Cargo"] - row["E/S"]) != row["F/S"] :
       print (f"Error en fila {idx}: {row['Cargo']} - {row['E/S']} = {(row['Cargo'] - row['E/S'])} != {row['F/S']}")


Error en fila 5: 37 - 20 = 17 != 0
Error en fila 23: 20 - 9 = 11 != 6
Error en fila 38: 1 - 0 = 1 != 0
Error en fila 55: 19 - 17 = 2 != 12
Error en fila 57: 14322 - 14322 = 0 != 1
Error en fila 115: 16 - 12 = 4 != 2
Error en fila 155: 35 - 22 = 13 != 14


## NORMALIZACIÓN DE LAS FALLAS

In [97]:
# Crear un nuevo DataFrame con columnas NNE y PRINCIPALES FALLAS
fallas_resumido = df_fallas[["INE", "PRINCIPALES FALLAS"]].copy()


In [98]:
fallas_resumido.head(40)

,INE,PRINCIPALES FALLAS
1,EQ DE TEN Y REPL RL 31 P/CABLE TEF CAMP,NaN
2,EJE DE CARRETE DE TEN Y REPL RL 27 D P/CABLE T...,NaN
3,EQ DE TEN Y REPL RL 26 P/CABLE TEF CAMP,UM (CUENTA MUSEO ) (ESC COM BL 1)
4,TELEFONO DE CAMPAÑA PORTATIL TA 1/PT,UM (CUENTA MUSEO ) (ESC COM BL 1)
5,TELEFONO DE CAMPAÑA PORTATIL TA 312 PT,PERILLA DE VOLUMEN CANT: OCHO (08) DETERIORADA...
6,CONMUTADOR TELEFONICO DE CAMPAÑA 12 LINEAS SB ...,UM (CUENTA MUSEO ) (ESC COM BL 1)
7,CONMUTADOR TELEFONICO DE CAMPAÑA 10 LIN-OB10-M...,UM (CUENTA MUSEO ) (ESC COM BL 1)
8,"CENTRO CONMUTADOR,MOVIL INTEGRADO.(CCMI)-C/PRONT",UPS (F/S) A DETERMINAR POR EL ESPECIALISTA. (E...
9,TELEFONO DE CAMPANA PORTATIL TP 9,UM (CUENTA MUSEO ) (ESC COM BL 1)
10,FAX - MOD KXFT982 - PANASONIC,NaN


### Eliminar las fallas que no admiten reparación o efectos que no presentan fallas

In [99]:
# despliegue la columna principales fallas en varias filas pero manteniendo la columna INE por cada parte de la lista de las filas 
fallas_resumido = fallas_resumido.explode("PRINCIPALES FALLAS")

# Eliminar las filas con valores NaN en la columna "PRINCIPALES FALLAS"
fallas_resumido = fallas_resumido.dropna(subset=["PRINCIPALES FALLAS"])


In [100]:


# ------------------------------
# 1) Funciones de normalización
# ------------------------------

def to_upper(s: pd.Series) -> pd.Series:
    return s.str.upper()

def strip_spaces(s: pd.Series) -> pd.Series:
    return s.str.strip()

def normalize_spaces(s: pd.Series) -> pd.Series:
    s = s.str.replace(r"\s+", " ", regex=True)
    s = s.str.replace(r"\(\s+", "(", regex=True)
    s = s.str.replace(r"\s+\)", ")", regex=True)
    s = s.str.replace(r"\s*,\s*", ", ", regex=True)
    s = s.str.replace(r"\s*-\s*", " - ", regex=True)
    s = s.str.replace(r"\s*:\s*", ": ", regex=True)
    return s

# Regex para split (no corta fechas ni F/S o E/S)
regex_split = r'(?<!\d)-(?!\d)|(?<!\d)(?<!F)(?<!E)/(?!\d)|\s+Y\s+|\s*,\s*'

def split_and_clean(s: pd.Series) -> pd.Series:
    return s.apply(
        lambda x: [p.strip() for p in re.split(regex_split, x) if p.strip()] if pd.notna(x) else []
    )

# ------------------------------
# 2) Pipeline
# ------------------------------

normalizer = Pipeline(steps=[
    ("to_upper", FunctionTransformer(to_upper)),
    ("strip_spaces", FunctionTransformer(strip_spaces)),
    ("normalize_spaces", FunctionTransformer(normalize_spaces)),
    ("split", FunctionTransformer(split_and_clean, validate=False)),
])

# ------------------------------
# 3) Uso sobre tu DataFrame
# ------------------------------

fallas_resumido["PRINCIPALES FALLAS"] = normalizer.fit_transform(fallas_resumido["PRINCIPALES FALLAS"])
#fallas_resumido["INE"] = normalizer.fit_transform(fallas_resumido["INE"])



In [101]:
# despliegue la columna principales fallas en varias filas pero manteniendo la columna INE por cada parte de la lista de las filas 
fallas_resumido = fallas_resumido.explode("PRINCIPALES FALLAS")

In [102]:
# Guardar el DataFrame en un nuevo archivo Excel
fallas_resumido.to_excel("resumen_fallas_ppl.xlsx", index=False)

In [ ]:
# Contar frecuencia de fallas

conteo_fallas = fallas_resumido.value_counts()

print("Fallas declaradas:")
n=1
for falla, cantidad in conteo_fallas.items():
    print(f" {n}) {falla}: {cantidad}")
    n += 1

# Guardar el conteo en un nuevo archivo Excel
conteo_fallas_df = conteo_fallas.reset_index(name='Cantidad')
conteo_fallas_df.to_excel("fallas.xlsx", index=False)

Fallas declaradas:
 1) ('EQUIPO DE RADIO PRC 4620', 'MICROTELEFONO H189 INOPERABLE'): 5
 2) ('EQUIPO DE RADIO.SEL 180/190(3W 40W)-30.A79.975MHZ-AURIC/LARI', 'S'): 2
 3) ('EQUIPO DE RADIO.SEL 180/190(3W 40W)-30.A79.975MHZ-AURIC/LARI', 'NI ENTREGADO A U3449 (L3) SEGÚN SRE137 NOTA RECIBO 24/062825'): 2
 4) ('BATERIA.ALI/107-14,4 V 4 A-C/CARG C/10-P/EQ DE RADIO TRC 300', 'SIETE (7) CAPSULA RX'): 1
 5) ('BATERIA BB - 4600 P/EQ RADIO PRC 3620 - 4620', '8 F/S NO RETIENEN CARGA'): 1
 6) ('CABINA C/EQUIPO RADIO GRC 9600 H-C/PRONT', 'OM NRO 2308019 FECHA 31/05/23 U3 SINTONIZADOR DE ANTENA (F/S) (REMITIDO B MANT COM 601)'): 1
 7) ('CABINA P/EQ RADIOENLACE DIGITAL DE CAMP. RDC C/PRONT', 'ROUTER F/S REMITIDO AL B MANT COM 601'): 1
 8) ('CABINA P/EQ RADIOENLACE DIGITAL DE CAMP. RDC C/PRONT', 'SRE 137 25/001611 (ESC COM BL 1)'): 1
 9) ('BATERIA.ALI/107-14,4 V 4 A-C/CARG C/10-P/EQ DE RADIO TRC 300', 'TX DEFICIENTES (GAA MIX 602)'): 1
 10) ('   KIT DE COMUNICACIONES USO EQ RADIOELECT VHF DE 2 COMPONENT